In [60]:
import numpy as np
import pandas as pd

from bokeh.io import show , output_notebook
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, Dropdown, CheckboxGroup, Button, Slider, MultiSelect
from bokeh.models.callbacks import CustomJS
from bokeh.layouts import column, gridplot, row
from bokeh.server.server import Server

def twb_app(doc):
   # output_notebook()

    #data_dir = '/home/antonis/TWB_translators/'
    #metadata = pd.read_excel(data_dir + 'Hackathon-for-Good-2019_TWB-Challenge_Metadata.xlsx')
    metadata = pd.read_pickle('data/english_document_data_updated.pkl')
    source = ColumnDataSource(metadata)
    
    extra_features = pd.read_pickle('data/freq_content_updated.pkl')
    metadata['Age'] = extra_features['Age']
    metadata['Gender'] = extra_features['Gender']
    metadata['Disabled'] = extra_features['disabl '] > 0.001
    
    columns = []
    for i in metadata.columns:
        temp = TableColumn(field=i, title=i)
        columns.append(temp)

    data_table = DataTable(source=source, columns=columns, width=1500, height=800)

    # click handler for Age variable
    def dropdown_change(attr, old, new):
        # find appropriates data rows based on new value
        new_data_view = metadata.loc[metadata['Age'] == menu[int(new)][0]]
        # update source of data table
        source = ColumnDataSource(new_data_view)
        new_data_table = DataTable(source=source, columns=columns, width=1500, height=800)
        # update the layout of the doc with new data table
        col.children[1] = new_data_table

    menu = [('0-18', '0'), ('18-65', '1'), ('65-', '2')]
    dropdown = Dropdown(label='Age', button_type='warning', menu=menu)
    dropdown.on_change('value', dropdown_change)

    def dropdown2_change(attr, old, new):
        new_data_view = metadata.loc[metadata['Gender'] == menu2[int(new)][0]]
        source = ColumnDataSource(new_data_view)
        new_data_table = DataTable(source=source, columns=columns, width=1500, height=800)
        col.children[1] = new_data_table
         
    menu2 = [('Men', '0'), ('Women', '1')]
    dropdown2 = Dropdown(label='Gender', button_type='warning', menu=menu2)
    dropdown2.on_change('value', dropdown2_change)
    
    def dropdown_change3(attrname, old, new):
        # find appropriates data rows based on new value
        #vcomp = multi_select.value
        new_data_view = metadata.loc[(metadata['Source_country'] == menu3[int(new)][0])]
        # update source of data table
        source = ColumnDataSource(new_data_view)
        new_data_table = DataTable(source=source, columns=columns, width=1500, height=800)
        # update the layout of the doc with new data table
        col.children[1] = new_data_table

    temp=metadata.Source_country.unique()
    temp=sorted(temp)
    menu3=[]
    for i in range(len(temp)):
        menu3.append((temp[i], str(i)))
        #menu3.append(temp[i])

    dropdown3 = Dropdown(label='Source country', button_type='warning', menu=menu3)
    #multi_select = MultiSelect(title="Source Country:", value=["GR"],options=menu3)
    dropdown3.on_change('value', dropdown_change3)
    
    def dropdown_change4(attrname, old, new):
        # find appropriates data rows based on new value
        #vcomp = multi_select.value
        new_data_view = metadata.loc[(metadata['Target_country'] == menu3[int(new)][0])]
        # update source of data table
        source = ColumnDataSource(new_data_view)
        new_data_table = DataTable(source=source, columns=columns, width=1500, height=800)
        # update the layout of the doc with new data table
        col.children[1] = new_data_table


    dropdown4 = Dropdown(label='Target country', button_type='warning', menu=menu3)
    #multi_select = MultiSelect(title="Source Country:", value=["GR"],options=menu3)
    dropdown4.on_change('value', dropdown_change4)
    
    def dropdown_change5(attrname, old, new):
        # find appropriates data rows based on new value
        #vcomp = multi_select.value
        new_data_view = metadata.loc[(metadata['Format'] == menu4[int(new)][0])]
        # update source of data table
        source = ColumnDataSource(new_data_view)
        new_data_table = DataTable(source=source, columns=columns, width=1500, height=800)
        # update the layout of the doc with new data table
        col.children[1] = new_data_table

    temp=metadata.Format.unique()
    temp=sorted(temp)
    menu4=[]
    for i in range(len(temp)):
        menu4.append((temp[i], str(i)))    

    dropdown5 = Dropdown(label='Format', button_type='warning', menu=menu4)
    #multi_select = MultiSelect(title="Source Country:", value=["GR"],options=menu3)
    dropdown5.on_change('value', dropdown_change5)

    # This works by returning a list of the checked checkboxes' indexes after a click
    def checkbox_change(new):
        # naive way of doing the changes
        '''
        if len(new) == 0:
            print("Gamieste")
            new_data_view = metadata.loc[(metadata['Disabled person'] == False) & (metadata['Unspecified'] == False)]
        elif len(new) == 1:
            new_data_view = metadata.loc[(metadata[new[0]] == True) & (metadata[['Disabled person', 'Unspecified'].remove(new[0])] == False)]
        else:
            new_data_view = metadata.loc[(metadata[new[0]] == True) & (metadata[new[1]] == True)]
        '''
        if len(new) == 0:
            new_data_view = metadata.loc[metadata['Disabled person'] == False]
        else:
            new_data_view = metadata.loc[metadata['Disabled person'] == True]
        source = ColumnDataSource(new_data_view)
        data_table = DataTable(source=source, columns=columns, width=1500, height=800)
    
        #col.children[1] = new_data_table

    #checkbox_group = CheckboxGroup(labels=['Disabled person','Unspecified'], active=[0, 1])
    checkbox_group = CheckboxGroup(labels=['Disabled person'])
    checkbox_group.on_click(checkbox_change)

    # this is confirmed to work
    def table_reset(new):
        col.children[1] = data_table

    button = Button(label="Reset", button_type="success")
    button.on_click(table_reset)
    
    def slider_weight_1_change(attrname, old, new):
        slider_weight_3.value=slider_weight_1.value
        #slider_weight_2.value=slider_weight_1.value
        
    slider_weight_1 = Slider(start=0, end=1, value=0.5, step=.1, title="Relevancy_to_news_weight")
    slider_weight_1.on_change('value', slider_weight_1_change)

    def slider_weight_2_change(attrname, old, new):
        slider_weight_1.value=slider_weight_2.value
        #slider_weight_3.value=slider_weight_2.value
        
        
    slider_weight_2 = Slider(start=0, end=1, value=0.5, step=.1, title="Document_features_weight")
    slider_weight_2.on_change('value', slider_weight_2_change)

    def slider_weight_3_change(attrname, old, new):
        #slider_weight_1.value=slider_weight_3.value
        slider_weight_2.value=slider_weight_3.value
        
    slider_weight_3= Slider(start=0, end=1, value=0.5, step=.1, title="Document_content_weight")
    slider_weight_3.on_change('value', slider_weight_3_change)
    
    col = row(column(dropdown, dropdown2, dropdown3, dropdown4, dropdown5,checkbox_group,button, slider_weight_1,slider_weight_2,slider_weight_3 ), data_table)
    doc.add_root(col)

In [53]:
'''
data_dir = '/home/antonis/TWB_translators/'
metadata2 = pd.read_excel(data_dir + 'Hackathon-for-Good-2019_TWB-Challenge_Metadata.xlsx')


temp=metadata2.Source_country.unique()
temp=sorted(temp)
menu3=[]
for i in range(len(temp)):
    menu3.append((temp[i], str(i)))
    #menu3.append(temp[i])

new_data_view = metadata2.loc[(metadata2['Source_country'] == menu3[int('4')][0])]
new_data_view.head()
'''


"\ndata_dir = '/home/antonis/TWB_translators/'\nmetadata2 = pd.read_excel(data_dir + 'Hackathon-for-Good-2019_TWB-Challenge_Metadata.xlsx')\n\n\ntemp=metadata2.Source_country.unique()\ntemp=sorted(temp)\nmenu3=[]\nfor i in range(len(temp)):\n    menu3.append((temp[i], str(i)))\n    #menu3.append(temp[i])\n\nnew_data_view = metadata2.loc[(metadata2['Source_country'] == menu3[int('4')][0])]\nnew_data_view.head()\n"

In [54]:
# runs the app using bokeh server unsing 1 thread in port 5007
server = Server({'/': twb_app}, num_procs=1, port=8080)
server.start()
server.show('/')

In [59]:
server.stop()

In [ ]:
'''
import numpy as np
import pandas as pd

from bokeh.io import show , output_notebook
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, Dropdown, CheckboxGroup, Button, Slider, MultiSelect
from bokeh.models.callbacks import CustomJS
from bokeh.layouts import column, gridplot, row
from bokeh.server.server import Server

output_notebook()   

data_dir = '/home/antonis/TWB_translators/'
metadata = pd.read_excel(data_dir + 'Hackathon-for-Good-2019_TWB-Challenge_Metadata.xlsx')

def dropdown_change3(attr,old,new):
    # find appropriates data rows based on new value
    #vcomp = multi_select.value
    print(dropdown3.value)
    new_data_view = metadata.loc[metadata['Source_country'] == menu3[int(dropdown3.value)][0]]
    # update source of data table
    source = ColumnDataSource(new_data_view)
    new_data_table = DataTable(source=source, columns=columns, width=1500, height=800)
    # update the layout of the doc with new data table
    col.children[1] = new_data_table


source = ColumnDataSource(metadata)
    
temp=metadata.Source_country.unique()
temp=sorted(temp)
menu3=[]
for i in range(len(temp)):
    menu3.append((temp[i], temp[i]+str(i)))
    #menu3.append(temp[i])

dropdown3 = Dropdown(label='Source country', button_type='warning', menu=menu3)
#multi_select = MultiSelect(title="Source Country:", value=["GR"],options=menu3)
dropdown3.on_change('value', dropdown_change3)

columns = []
for i in metadata.columns:
    temp = TableColumn(field=i, title=i)
    columns.append(temp)

data_table = DataTable(source=source, columns=columns, width=500, height=200)

col = row(column(dropdown3), data_table)
show(col)
'''

In [62]:
metadata = pd.read_pickle('data/english_document_data.pkl')
source = ColumnDataSource(metadata)
    
extra_features = pd.read_pickle('data/freq_content_updated.pkl')
metadata['Age'] = extra_features['Age']
metadata['Gender'] = extra_features['Gender']
metadata['Disabled'] = extra_features['disabl '] > 0.001

In [58]:
extra_features['disabl ']

0        0.000000
1        0.000000
2        0.000000
3        0.000000
4        0.000000
5        0.000000
6        0.000000
7        0.000000
8        0.000000
9        0.000000
10       0.000000
11       0.000000
12       0.000000
13       0.000000
14       0.000000
15       0.000000
16       0.000000
17       0.000000
18       0.000000
19       0.000135
20       0.000135
21       0.000135
22       0.000343
23       0.000343
24       0.000343
25       0.000343
26       0.000343
27       0.000343
28       0.000343
29       0.000000
           ...   
34495    0.002427
34496    0.002427
34497    0.000000
34498    0.000000
34499    0.000000
34500    0.000000
34501    0.000000
34502    0.000000
34503    0.000000
34504    0.000000
34505    0.000000
34506    0.000000
34507    0.000000
34508    0.000000
34509    0.000000
34510    0.000000
34511    0.000000
34512    0.000000
34513    0.000000
34514    0.000000
34515    0.000000
34516    0.000000
34517    0.000000
34518    0.000000
34519    0